In [31]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from tensorboardX import SummaryWriter
import tqdm
import os
import uuid

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

unique_id = str(uuid.uuid4())

def weights_initialize(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
        module.bias.data.fill_(0.01)
        
class _TransModel(nn.Module):
    """ Model for DQN """

    def __init__(self, input_len, output_len):
        super(_TransModel, self).__init__()
        
        self.fc1 = nn.Sequential(
            torch.nn.Linear(input_len, 1024),
            torch.nn.BatchNorm1d(1024),
            nn.ReLU()
        )
        self.fc1.apply(weights_initialize)
        
        self.fc2 = nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            nn.ReLU()
        )
        self.fc2.apply(weights_initialize)
        
        self.fc3 = nn.Sequential(
            torch.nn.Linear(512, 128),
            torch.nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.fc3.apply(weights_initialize)
        
        self.output_layer = nn.Sequential(
            torch.nn.Linear(128, output_len)
        )
        self.output_layer.apply(weights_initialize)
        
    def forward(self, input):
        x = self.fc1(input)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return self.output_layer(x)

    
class TransModel():
    def __init__(self, input_len, ouput_len, learning_rate = 0.0001):
        self.model = _TransModel(input_len, ouput_len)
        
        if use_cuda:
            print("Using GPU")
            self.model = self.model.cuda()
        else:
            print("Using CPU")
        self.steps = 0
        # self.model = nn.DataParallel(self.model)
        self.optimizer = Adam(self.model.parameters(), lr = learning_rate)
        self.loss_fn = nn.MSELoss(reduction='mean')
        
        self.steps = 0
        
    def predict(self, input, steps, learning):
        output = self.model(input).squeeze(1)
        #reward, next_state = output[0], output[1:]

        return output

    def predict_batch(self, input):
        output = self.model(input)
        #reward, next_state = output[:, 0], output[:, 1:]
        return output

    def fit(self, state, target_state):
        loss = self.loss_fn(state, target_state)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.steps += 1
        return loss
    
    def save(self):
        cwd = os.getcwd()
        path = cwd + '/models'
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
        file_path = path + '/UNITS_' + unique_id + '.pt'
        torch.save(self.model.state_dict(), file_path)

In [32]:
#data = torch.load('test_random_vs_random_2l.pt')
data = torch.load('all_experiences_100000.pt')

np.set_printoptions(suppress=True)
l = len(data)

norm_vector = np.array([700, # p1 minerals
                        50, 40, 20, 50, 40, 20, 3, # p1 top and bottom lane buildings
                        50, 40, 20, 50, 40, 20, 3, # p2 top and bottom lane buildings
                        1, 1, 1, 1, 1, 1, # p1 top and bottom lane units
                        1, 1, 1, 1, 1, 1, # p1 top and bottom lane units
                        2000, 2000, 2000, 2000, 40])

for i in range(0, len(data)):
    #data[i][0] = data[i][0] / norm_vector # normalize input
    #data[i][1] = data[i][1][0:32] / norm_vector # normalize ground truth
    data[i][0] = data[i][0]
    data[i][1] = data[i][1][0:32]

    data[i][1] = data[i][1][15:27] # set output to units in top and bottom lanes 
    
#print(data[0][0])
#print(data[0][1])

np.random.shuffle(data)

train_data = np.array(data[: int(np.floor(l * 0.5))])
test_data = np.array(data[int(np.floor(l * 0.5)) : ])
print(train_data.shape, test_data.shape)

batch_size = 64
summary_test = SummaryWriter(log_dir = 'units-on-field-transition-model-report/')

(50000, 2) (50000, 2)


In [33]:
baselineCurrent = np.stack(test_data[:, 0])
baselineNextUnits = np.stack(test_data[:, 1])
baselineCurrentUnits = baselineCurrent[:, 15:27]

print(baselineCurrentUnits[0])
print(baselineNextUnits[0])

mse_baseline = ((baselineCurrentUnits - baselineNextUnits)**2).mean(axis=None)
print(mse_baseline)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.95622


In [34]:
trans_model = TransModel(len(data[0][0]), len(data[0][1]))

Using GPU


In [35]:
def evaluation(model, data, epoch):
    state_action = torch.from_numpy(np.stack(data[:, 0])).type(FloatTensor)
    next_state_reward = torch.from_numpy(np.stack(data[:, 1])).type(FloatTensor)
    
    model.model.eval()
    criterion = nn.MSELoss(reduction='mean')
    outputs = model.predict_batch(state_action)
    mse = criterion(outputs, next_state_reward)
    mse_p1 = criterion(outputs, next_state_reward)
    mse_p2 = criterion(outputs, next_state_reward)

    accuracy = torch.sum( torch.sum( torch.eq( outputs, next_state_reward ) )).item()
    accuracy = accuracy / (2 * outputs.size()[0])

    model.model.train()
    
    summary_test.add_scalar("MSE", float(mse.item()), epoch)
    summary_test.add_scalars("MSE",{'Player 1 Unit MSE': float(mse_p1.item())}, epoch)
    summary_test.add_scalars("MSE",{'Player 2 Unit MSE': float(mse_p2.item())}, epoch)
    summary_test.add_scalars("MSE",{'Baseline Unit MSE': float(mse_baseline)}, epoch)

    #summary_test.add_scalar(tag="Accuracy (Correct / Total)",
    #                        scalar_value=float(accuracy),
    #                        global_step=epoch)
    
    f = open("units-on-field-transition-model-report/test_loss.txt", "a+")
    f.write("loss:" + str(mse.item()) + ", ")
    f.write("acc:" + str(accuracy) + "\n")
    if epoch % 1000 == 0:
        f.write("output:" + str(outputs[0:2]) + "\n")
        f.write("ground true:" + str(next_state_reward[0:2]) + "\n")
    f.close()
    return mse.item()

In [ ]:
state_action = torch.from_numpy(np.stack(train_data[:, 0])).type(FloatTensor)
next_state_reward = torch.from_numpy(np.stack(train_data[:, 1])).type(FloatTensor)
print(state_action.size(), next_state_reward.size())

for epoch in tqdm.tqdm(range(10000)):
    loss = 0
    s = np.arange(state_action.shape[0])
    np.random.shuffle(s)
    train_x = state_action[s]
    train_y = next_state_reward[s]
    for i in range(state_action.shape[0] // batch_size + 1):
        if (i + 1) * batch_size <= state_action.shape[0]:
            start = i * batch_size
            end = (i + 1) * batch_size
        else:
            start = i * batch_size
            end = state_action.shape[0]
        #print(start, end)
        inputs, ground_true = train_x[start : end, :], train_y[start : end, :]
        outputs = trans_model.predict_batch(inputs)
        loss += trans_model.fit(outputs, ground_true)
#     print(epoch)
    summary_test.add_scalars("MSE",{'Train MSE': float(loss / (state_action.shape[0] // batch_size + 1) )}, epoch)
    evaluation(trans_model, test_data, epoch)
    #break

#trans_model.save()



  0%|          | 0/10000 [00:00<?, ?it/s]

torch.Size([50000, 32]) torch.Size([50000, 12])




  0%|          | 1/10000 [00:02<5:48:31,  2.09s/it]

  0%|          | 2/10000 [00:04<5:48:19,  2.09s/it]

  0%|          | 3/10000 [00:06<5:47:55,  2.09s/it]

  0%|          | 4/10000 [00:08<5:49:05,  2.10s/it]

  0%|          | 5/10000 [00:10<5:52:12,  2.11s/it]

  0%|          | 6/10000 [00:12<5:50:23,  2.10s/it]

  0%|          | 7/10000 [00:14<5:47:54,  2.09s/it]

  0%|          | 8/10000 [00:16<5:47:03,  2.08s/it]

  0%|          | 9/10000 [00:18<5:54:16,  2.13s/it]

  0%|          | 10/10000 [00:21<5:52:06,  2.11s/it]

  0%|          | 11/10000 [00:23<5:47:45,  2.09s/it]

  0%|          | 12/10000 [00:25<5:48:58,  2.10s/it]

  0%|          | 13/10000 [00:27<5:49:57,  2.10s/it]

  0%|          | 14/10000 [00:29<5:48:30,  2.09s/it]

  0%|          | 15/10000 [00:31<5:47:41,  2.09s/it]

  0%|          | 16/10000 [00:33<5:47:39,  2.09s/it]

  0%|          | 17/10000 [00:35<5:45:57,  2.08s/it]

  0%|          | 18/10000 [00:37<5:44:39,  2.07s/it]

  0%|          | 19/10000 [00:39<5: